<a href="https://colab.research.google.com/github/ShangLin-Wu/Speech-to-text/blob/main/%E3%80%8CSpeech_To_Text_version2-2_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mp3 to text with deep speech model

---

In [86]:
from __future__ import absolute_import, division, print_function

import os
import numpy as np
import shlex
import subprocess
import sys
import wave

!pip install deepspeech 
from deepspeech import Model, version
from timeit import default_timer as timer

# audio converters
!apt update && apt-get install ffmpeg mpg123

# sox package for adjusting sample rate.
!apt-get install libsox-fmt-all libsox-dev sox

# neural network model for acoustic recognition
# !wget -O - https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/models_0.9.tar.gz | tar xvfz -

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

# Accoustic parameters

In [205]:
model    = '/content/models/deepspeech-0.9.3-models.pbmm'
scorer = '/content/models/deepspeech-0.9.3-models.scorer'

# These constants control the beam search decoder

# Beam width used in the CTC decoder when building candidate transcriptions
BEAM_WIDTH = 100

# The alpha hyperparameter of the CTC decoder. Language Model weight
LM_WEIGHT = 0.76

# Valid word insertion weight. This is used to lessen the word insertion penalty
# when the inserted word is part of the vocabulary
VALID_WORD_COUNT_WEIGHT = 1.8


# These constants are tied to the shape of the graph used (changing them changes
# the geometry of the first layer), so make sure you use the same constants that
# were used during training

# Number of MFCC features to use
# N_FEATURES = 32

# Size of the context window used for producing timesteps in the input vector
# N_CONTEXT = 12

# Adapt Sample Rate of Audio File

In [178]:
def convert_samplerate(audio_path):
    sox_cmd = 'sox {} --type raw --bits 16 --channels 1 --rate 16000 --encoding signed-integer --endian little --compression 0.0 --no-dither - '.format(audio_path)
    try:
        output = subprocess.check_output(shlex.split(sox_cmd), stderr=subprocess.PIPE)
    except subprocess.CalledProcessError as e:
        raise RuntimeError('SoX returned non-zero status: {}'.format(e.stderr))
    except OSError as e:
        raise OSError(e.errno, 'SoX not found, use 16kHz files or install it: {}'.format(e.strerror))

    return 16000, np.frombuffer(output, np.int16)



# Input MP3 Audio File

In [173]:
# upload mp3 audio file.

# from google.colab import files
# uploaded = files.upload()
# for audio in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#          name=audio, length=len(uploaded[audio])))

# os.rename(audio, 'speech.mp3')
# audio = 'speech.wav'

# convert to wav file.  
# !ffmpeg -i speech.mp3 -vn -acodec pcm_s16le -ac 1 -ar 16000 -f wav speech.wav
# !mpg123 -w speech.wav speech.mp3

# Convert MP3 to Text

In [206]:
    audio = 'speech.wav'   
    
    print('Loading model from file {}'.format(model), file=sys.stderr)
    model_load_start = timer()
    ds = Model(model)
    model_load_end = timer() - model_load_start
    print('Loaded model in {:.3}s.'.format(model_load_end), file=sys.stderr)

    lm_load_start = timer()
    ds.setBeamWidth(BEAM_WIDTH)
    ds.enableExternalScorer(scorer)
    ds.setScorerAlphaBeta(LM_WEIGHT,VALID_WORD_COUNT_WEIGHT)
    lm_load_end = timer() - lm_load_start
        
    fin = wave.open(audio, 'rb')
    fs = fin.getframerate()

    if fs != 16000:
        print('Warning: original sample rate ({}) is different than 16kHz. Resampling might produce erratic speech recognition.'.format(fs), file=sys.stderr)
        fs, audio = convert_samplerate(audio)
    else:
        bf = fin.readframes(fin.getnframes())
        audio = np.frombuffer(bf, np.int16)

    audio_length = fin.getnframes() * (1/16000)
    fin.close()

    print('Running inference.', file=sys.stderr)
    print('================================\n')
    inference_start = timer()
    print(ds.stt(audio))
    inference_end = timer() - inference_start
    print('\n================================')
    print('Inference took %0.3fs for %0.3fs audio file.' % (inference_end, audio_length), file=sys.stderr)

Loading model from file /content/models/deepspeech-0.9.3-models.pbmm
Loaded model in 0.00977s.
Running inference.



she fantasies about traveling through the regions archeological sights and picturesque town ever since she was a teenager growing up in finland poring over travel brochures and daydreaming about future adventures



Inference took 7.952s for 12.696s audio file.


In [ ]:
# She'd fantasized about traveling through the region's archaeological sites and picturesque towns ever since she was a teenager growing up in Finland, poring over travel brochures and daydreaming about future adventures.

In [ ]:
# Beam width used in the CTC decoder when building candidate transcriptions
BEAM_WIDTH = 100

# The alpha hyperparameter of the CTC decoder. Language Model weight
LM_WEIGHT = 0.76

# Valid word insertion weight. This is used to lessen the word insertion penalty
# when the inserted word is part of the vocabulary
VALID_WORD_COUNT_WEIGHT = 1.8

# she fantasies about traveling through the regions archeological sights and picturesque town ever since she was a teenager growing up in finland poring over travel brochures and daydreaming about future adventures
